In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from dateutil.parser import parse
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
international_flight_2020 = pd.read_csv('data/international_flight_2020.csv')
national_flight_2020 = pd.read_csv('data/national_flight_2020.csv')

KeyboardInterrupt: 

In [ ]:
all_flight_2020 = pd.concat([international_flight_2020, national_flight_2020])

In [ ]:
international_flight_2019 = pd.read_csv('data/international_flight_2019.csv')
national_flight_2019 = pd.read_csv('data/national_flight_2019.csv')
all_flight_2019 = pd.concat([international_flight_2019, international_flight_2019])

In [5]:
all_flight_2019_groupby_date = all_flight_2019.groupby(by = ['day'], as_index=False).count()
all_flight_2019_groupby_date = all_flight_2019_groupby_date[['day', 'origin']]
all_flight_2019_groupby_date.columns = ['day', 'count']

# Convert string to date data type
all_flight_2019_groupby_date['day'] = all_flight_2019_groupby_date['day'].apply(lambda x: parse(x).date())
all_flight_2019_groupby_date['Date'] = all_flight_2019_groupby_date['day'].apply(lambda x: (x.strftime('%m-%d')))
all_flight_2019_groupby_date['Year'] = all_flight_2019_groupby_date['day'].apply(lambda x: (x.year))

destination_count = all_flight_2019['destination'].value_counts().to_dict()
origin_count = all_flight_2019['origin'].value_counts().to_dict()

import collections
Cdict = Counter(destination_count) + Counter(origin_count) 
sorted_dict = collections.OrderedDict(Cdict)


,number,typecode,origin,destination,day,origin_ident,origin_type,origin_name,origin_latitude_deg,origin_longitude_deg,...,origin_municipality,destination_ident,destination_type,destination_name,destination_latitude_deg,destination_longitude_deg,destination_continent,destination_iso_country,destination_iso_region,destination_municipality
0,NaN,F50,YPAD,WSSL,2020-01-01 00:00:00+00:00,YPAD,large_airport,Adelaide International Airport,-34.945000,138.531006,...,Adelaide,WSSL,medium_airport,Seletar Airport,1.416950,103.867996,AS,SG,SG-02,Seletar
1,MU771,A359,YSSY,EHAM,2020-01-01 00:00:00+00:00,YSSY,large_airport,Sydney Kingsford Smith International Airport,-33.946098,151.177002,...,Sydney,EHAM,large_airport,Amsterdam Airport Schiphol,52.308601,4.763890,EU,NL,NL-NH,Amsterdam
2,NaN,B789,SAEZ,LEBL,2020-01-01 00:00:00+00:00,SAEZ,large_airport,Ministro Pistarini International Airport,-34.822200,-58.535800,...,Buenos Aires,LEBL,large_airport,Barcelona International Airport,41.297100,2.078460,EU,ES,ES-CT,Barcelona
3,NaN,A332,RKSI,LIRF,2020-01-01 00:00:00+00:00,RKSI,large_airport,Incheon International Airport,37.469101,126.450996,...,Seoul,LIRF,large_airport,Leonardo da Vinci–Fiumicino Airport,41.800278,12.238889,EU,IT,IT-62,Rome
4,NaN,B788,SBGR,LOWW,2020-01-01 00:00:00+00:00,SBGR,large_airport,Guarulhos - Governador André Franco Montoro In...,-23.435556,-46.473057,...,São Paulo,LOWW,large_airport,Vienna International Airport,48.110298,16.569700,EU,AT,AT-9,Vienna (Schwechat)


In [159]:
airports = list(sorted_dict.keys())[:3000]

In [160]:
test_data = all_flight_2020[:10000]
test_data['day']  = test_data['day'].apply(lambda x: parse(x).date())
test_data['date'] = test_data['day'].apply(lambda x: (x.strftime('%Y-%m-%d')))

C:\Users\WEI YUAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\WEI YUAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [157]:
data_6_4 = test_data[test_data['date'] == '2020-06-09']

In [158]:
len(set(data_6_4["origin_ident"]))

0

In [161]:
def build_daily_graph(df, days, airports):
    graph_list = []
    for day in days:
        cur_day_flights = df[df['day'] == day]
        directed_G = nx.DiGraph()
        directed_G.add_nodes_from(airports)
#         for airport in airports:
#             directed_G.add_no de(airport)
        for index, row in cur_day_flights.iterrows():
            if directed_G.has_edge(row['origin'], row['destination']):
                directed_G.edges[row['origin'], row['destination']]['weight'] += 1
            else:
                directed_G.add_edge(row['origin'], row['destination'], weight=1)
        graph_list.append(directed_G)
    return graph_list

In [162]:
days = set(test_data['day'])

graph_list_2020 = build_daily_graph(test_data, days, airports)

In [163]:
days

{datetime.date(2020, 1, 1), datetime.date(2020, 1, 2)}

In [168]:
nx.number_strongly_connected_components(graph_list_2020[1])

2727

In [169]:
nx.number_of_isolates(graph_list_2020[1])

2465

In [109]:
def find_SCC_isolate_number(graph_list):
    number_SCC_list = []
    number_isolate_list = []
    for graph in graph_list:
        number_SCC_list.append(nx.number_strongly_connected_components(graph))
        number_isolate_list.append(nx.number_of_isolates(graph))
    return number_SCC_list, number_isolate_list

In [110]:
number_SCC_list, number_isolate_list = find_SCC_isolate_number(graph_list_2020)

In [111]:
number_SCC_list

[369, 417, 432, 425, 329, 425, 309]

In [112]:
number_isolate_list

[23, 23, 23, 23, 23, 23, 23]

In [113]:
all_flight_2020.shape

(6760527, 23)

In [114]:
test_data = all_flight_2020[6000000:6100000]
test_data['day']  = test_data['day'].apply(lambda x: parse(x).date())
test_data['date'] = test_data['day'].apply(lambda x: (x.strftime('%Y-%m-%d')))

C:\Users\WEI YUAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\WEI YUAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [115]:
days = set(test_data['day'])

graph_list_2020 = build_daily_graph(test_data, days, airports)

In [122]:
test_data

,number,typecode,origin,destination,day,origin_ident,origin_type,origin_name,origin_latitude_deg,origin_longitude_deg,...,destination_ident,destination_type,destination_name,destination_latitude_deg,destination_longitude_deg,destination_continent,destination_iso_country,destination_iso_region,destination_municipality,date
4524350,NaN,NaN,KPEX,KFCM,2020-06-04,KPEX,small_airport,Paynesville Municipal Airport,45.372299,-94.746597,...,KFCM,small_airport,Flying Cloud Airport,44.827202,-93.457100,NaN,US,US-MN,Minneapolis,2020-06-04
4524351,NaN,B737,KSJC,KLGB,2020-06-04,KSJC,large_airport,Norman Y. Mineta San Jose International Airport,37.362598,-121.929001,...,KLGB,medium_airport,Long Beach Airport (Daugherty Field),33.817699,-118.152000,NaN,US,US-CA,Long Beach,2020-06-04
4524352,NaN,LJ60,KCRG,KMIA,2020-06-04,KCRG,medium_airport,Jacksonville Executive at Craig Airport,30.336300,-81.514397,...,KMIA,large_airport,Miami International Airport,25.793200,-80.290604,NaN,US,US-FL,Miami,2020-06-04
4524353,NaN,SW4,KLAS,KPHX,2020-06-04,KLAS,large_airport,McCarran International Airport,36.080101,-115.152000,...,KPHX,large_airport,Phoenix Sky Harbor International Airport,33.434299,-112.012001,NaN,US,US-AZ,Phoenix,2020-06-04
4524354,NaN,G2CA,74OR,74OR,2020-06-04,74OR,small_airport,Stan Jost Airport,45.323700,-123.018997,...,74OR,small_airport,Stan Jost Airport,45.323700,-123.018997,NaN,US,US-OR,Newberg,2020-06-04
4524355,NaN,NaN,KCGZ,KCHD,2020-06-04,KCGZ,small_airport,Casa Grande Municipal Airport,32.954899,-111.766998,...,KCHD,small_airport,Chandler Municipal Airport,33.269100,-111.810997,NaN,US,US-AZ,Chandler,2020-06-04
4524356,NaN,NaN,KSFO,KLAX,2020-06-04,KSFO,large_airport,San Francisco International Airport,37.618999,-122.375000,...,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,NaN,US,US-CA,Los Angeles,2020-06-04
4524357,NaN,C560,KVNY,KRNO,2020-06-04,KVNY,medium_airport,Van Nuys Airport,34.209801,-118.489998,...,KRNO,large_airport,Reno Tahoe International Airport,39.499100,-119.767998,NaN,US,US-NV,Reno,2020-06-04
4524358,NaN,NaN,CA84,KOKB,2020-06-04,CA84,small_airport,Torrey Pines Gliderport,32.889702,-117.247002,...,KOKB,small_airport,Oceanside Municipal Airport,33.217300,-117.353996,NaN,US,US-CA,Oceanside,2020-06-04
4524359,NaN,NaN,PANC,44AK,2020-06-04,PANC,large_airport,Ted Stevens Anchorage International Airport,61.174400,-149.996002,...,44AK,small_airport,West Papoose Lake Airpark,61.533401,-150.102005,NaN,US,US-AK,Wasilla,2020-06-04


In [117]:
number_SCC_list, number_isolate_list = find_SCC_isolate_number(graph_list_2020)

In [118]:
number_SCC_list

[3062, 1131, 2986, 3161, 3146]

In [119]:
number_isolate_list

[23, 23, 23, 23, 23]